In this notebook I precompute the coarse flow predicted with GMFlow model.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../src')

In [3]:
import torch

from torch.utils.data import DataLoader
from data.scannet.utils_scannet import ScanNetDataset

from matching.gmflow.gmflow.gmflow import GMFlow 
from utils.model import load_checkpoint

from tqdm.auto import tqdm
import wandb

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
batch_size = 3

### 1. Data

In [10]:
train_data = ScanNetDataset(
    root_dir='/home/project/data/ScanNet/scans/',
    npz_path='/home/project/ScanNet/smart_sample_train.npz',
    intrinsics_path='/home/project/ScanNet/scannet_indices/intrinsics.npz',
    mode='train'
)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False, drop_last=False, pin_memory=True, num_workers=0)

val_data = ScanNetDataset(
    root_dir='/home/project/data/ScanNet/scans/',
    npz_path='/home/project/code/data/scannet_splits/smart_sample_val.npz',
    intrinsics_path='/home/project/ScanNet/scannet_indices/intrinsics.npz',
    mode='val'
)

val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, drop_last=False, pin_memory=True, num_workers=0)


### 2. Model

In [7]:
checkpoint = load_checkpoint('/home/project/code/src/matching/gmflow/weights/pretrained/gmflow_with_refine_kitti-8d3b9786.pth',
                             device)

In [8]:
model = GMFlow(
    num_scales=2,
    upsample_factor=4,
    feature_channels=128,
    attention_type='swin',
    num_transformer_layers=6,
    ffn_dim_expansion=4,
    num_head=1,
    fine_tuning=True
)

model.load_state_dict(checkpoint['model'])
model.to(device);


for p in model.parameters():
    p.requires_grad = False
    
model.eval();

### 3. Train pairs

In [8]:
path_to_save = '/home/project/coarse_flow/train/'

with torch.no_grad():
    for data in tqdm(train_loader):
        data['image0'] = data['image0'].to(device)
        data['image1'] = data['image1'].to(device)
        out = model(
                data['image0'],
                data['image1'],
                attn_splits_list=[2, 8],
                corr_radius_list=[-1, 4],
                prop_radius_list=[-1, 1]
               )
        
        for i in range(data['image0'].size(0)):
            torch.save(out['flow_preds'][0][i].float().detach().cpu(), f'{path_to_save}{data["pair_name"][i]}.pt')

/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


KeyboardInterrupt: 

In [11]:
path_to_save = '/home/project/coarse_flow/val/'

with torch.no_grad():
    for data in tqdm(val_loader):
        data['image0'] = data['image0'].to(device)
        data['image1'] = data['image1'].to(device)
        out = model(
                data['image0'],
                data['image1'],
                attn_splits_list=[2, 8],
                corr_radius_list=[-1, 4],
                prop_radius_list=[-1, 1]
               )
        
        for i in range(data['image0'].size(0)):
            torch.save(out['flow_preds'][0][i].float().detach().cpu(), f'{path_to_save}{data["pair_name"][i]}.pt')

  0%|                                                                                                                             | 0/13334 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 13333/13334 [1:52:08<00:00,  1.98it/s]


IndexError: index 1 is out of bounds for dimension 0 with size 1